In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

rotten_ds = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [ ]:
print(rotten_ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [ ]:
import pandas as pd
# Convert the dataset to a pandas DataFrame for easier exploration
train_df = pd.DataFrame(rotten_ds['train'])
validation_df = pd.DataFrame(rotten_ds['validation'])
test_df = pd.DataFrame(rotten_ds['test'])

In [ ]:
# Display the first few rows of each DataFrame
print("Train set:")
print(train_df.head())

print("\nValidation set:")
print(validation_df.head())

print("\nTest set:")
print(test_df.head())

Train set:
                                                text  label
0  the rock is destined to be the 21st century's ...      1
1  the gorgeously elaborate continuation of " the...      1
2                     effective but too-tepid biopic      1
3  if you sometimes like to go to the movies to h...      1
4  emerges as something rare , an issue movie tha...      1

Validation set:
                                                text  label
0  compassionately explores the seemingly irrecon...      1
1  the soundtrack alone is worth the price of adm...      1
2  rodriguez does a splendid job of racial profil...      1
3  beneath the film's obvious determination to sh...      1
4    bielinsky is a filmmaker of impressive talent .      1

Test set:
                                                text  label
0  lovingly photographed in the manner of a golde...      1
1              consistently clever and suspenseful .      1
2  it's like a " big chill " reunion of the baade...      1
3

In [26]:
# Display the distribution of labels in the train set
print("\nLabel Distribution in Train Set:")
print(train_df['label'].value_counts())


Label Distribution in Train Set:
label
1    4265
0    4265
Name: count, dtype: int64


In [27]:
# Display the distribution of labels in the validation set
print("\nLabel Distribution in validation Set:")
print(validation_df['label'].value_counts())


Label Distribution in validation Set:
label
1    533
0    533
Name: count, dtype: int64
